In [89]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
from pytrends.request import TrendReq
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
today = datetime.today()
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [41]:
from ipynb.fs.full.ML import *

In [42]:
script = "9ho5HG7o00PT-g"
secret = "2CQTFbYyYp5aLEN7bHkKGO8X4E3YHQ"

In [43]:
posts = pd.read_csv("trimmed_posts.csv")

In [44]:
selected_cols = ['title','selftext']
ticker = 'MSFT'

In [45]:
try:
    new_df = posts[posts[selected_cols].apply(lambda x: x.str.contains(ticker)).all(axis=1)]
    data = new_df
    data['Ticker'] = ticker
    group_df = new_df.groupby('created_utc') \
       .agg({'id':'count', 'num_comments':'mean', 'score':'mean'}) \
       .rename(columns={'id':'post_count','num_comments':'avg_comments', 'score':'avg_score'}) \
       .reset_index()
    group_df['Ticker'] = ticker
except:
    places = {'created_utc': 0, 'post_count': 0, 'avg_comments': 0, 'avg_score': 0, 'ticker': ticker}
    group_df = pd.DataFrame.from_dict(places)

<ipython-input-45-7c11cac11b67>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Ticker'] = ticker


In [46]:
group_df

,created_utc,post_count,avg_comments,avg_score,Ticker
0,2020-12-08,1,4.000000,1.00,MSFT
1,2020-12-09,3,2.666667,1.00,MSFT
2,2020-12-10,1,17.000000,1.00,MSFT
3,2020-12-16,5,40.800000,4.60,MSFT
4,2020-12-17,4,26.250000,6.25,MSFT
5,2020-12-19,1,51.000000,1.00,MSFT
6,2020-12-21,2,12.500000,1.00,MSFT
7,2020-12-22,2,0.000000,1.00,MSFT
8,2020-12-24,1,0.000000,1.00,MSFT
9,2020-12-25,2,17.000000,1.00,MSFT


In [47]:
super_posts_df = data.loc[data['score'] > 300]
if not super_posts_df.empty:
    super_posts_df = super_posts_df.groupby(['created_utc', 'Ticker'])['score'].apply(lambda x: (x>=300).sum()).reset_index(name='Count_Score_300')

In [48]:
beginDate = '2020-12-01'
endDate = datetime.today().strftime('%Y-%m-%d')

try:
    pytrend = TrendReq(hl='en-US', tz=360)
    pytrend.build_payload(kw_list=[ticker], timeframe=beginDate + ' ' + endDate, geo='US')
    df = pytrend.interest_over_time()
    df['Noise'] = df[ticker]
    df[ticker]= ticker
    df.index.names = ['Date']
    df.columns = ['Ticker','isPartial', 'Noise']
    mergedNoise = df
except:
    noise = {'Ticker': ticker, 'isPartial': 'No', 'Noise': 0}
    mergedNoise = pd.DataFrame.from_dict(noise)

In [49]:
group_df['created_utc'] = pd.to_datetime(group_df['created_utc'])

if not super_posts_df.empty:
    super_posts_df['created_utc'] = pd.to_datetime(super_posts_df['created_utc'])

In [50]:
merged = group_df.merge(mergedNoise, left_on=['created_utc', 'Ticker'], right_on=['Date', 'Ticker'], how='left')
if not super_posts_df.empty:
    merged = merged.merge(super_posts_df, left_on=['created_utc', 'Ticker'], right_on=['created_utc', 'Ticker'], how="left")
else:
    merged['Count_Score_300'] = 0


In [51]:
merged.drop(columns=['isPartial'])

,created_utc,post_count,avg_comments,avg_score,Ticker,Noise,Count_Score_300
0,2020-12-08,1,4.000000,1.00,MSFT,25,0
1,2020-12-09,3,2.666667,1.00,MSFT,22,0
2,2020-12-10,1,17.000000,1.00,MSFT,27,0
3,2020-12-16,5,40.800000,4.60,MSFT,27,0
4,2020-12-17,4,26.250000,6.25,MSFT,23,0
5,2020-12-19,1,51.000000,1.00,MSFT,7,0
6,2020-12-21,2,12.500000,1.00,MSFT,29,0
7,2020-12-22,2,0.000000,1.00,MSFT,31,0
8,2020-12-24,1,0.000000,1.00,MSFT,19,0
9,2020-12-25,2,17.000000,1.00,MSFT,5,0


In [52]:
stockData = yf.download(ticker, start=beginDate, end=endDate)
stockData['Ticker'] = ticker
stockReport = pd.DataFrame(stockData, columns= ['Ticker','Adj Close','Volume'])
merged = merged.merge(stockReport, left_on=['created_utc', 'Ticker'], right_on=['Date', 'Ticker'], how='left')

[*********************100%***********************]  1 of 1 completed


In [53]:
merged['post_count_change'] = merged['post_count'].pct_change()
merged['avg_score_change'] = merged['avg_score'].pct_change()
merged['Adj Close_change'] = merged['Adj Close'].pct_change()
merged['Volume_change'] = merged['Volume'].pct_change()
merged.replace([np.inf, -np.inf], np.nan, inplace=True)
merged.replace([np.inf, -np.inf], np.nan).dropna(subset=['post_count_change', 'avg_score_change', 'Volume_change'], how="all")
merged = merged.dropna()

In [54]:
merged

,created_utc,post_count,avg_comments,avg_score,Ticker,isPartial,Noise,Count_Score_300,Adj Close,Volume,post_count_change,avg_score_change,Adj Close_change,Volume_change
1,2020-12-09,3,2.666667,1.00,MSFT,False,22,0,211.313309,32440600.0,2.000000,0.000000,-0.019490,0.393251
2,2020-12-10,1,17.000000,1.00,MSFT,False,27,0,210.036255,26733300.0,-0.666667,0.000000,-0.006043,-0.175931
3,2020-12-16,5,40.800000,4.60,MSFT,False,27,0,218.776123,35023300.0,4.000000,3.600000,0.041611,0.310100
4,2020-12-17,4,26.250000,6.25,MSFT,False,23,0,218.915802,32515800.0,-0.200000,0.358696,0.000638,-0.071595
6,2020-12-21,2,12.500000,1.00,MSFT,False,29,0,222.078506,37181900.0,1.000000,0.000000,0.014447,0.143503
7,2020-12-22,2,0.000000,1.00,MSFT,False,31,0,223.425415,22612200.0,0.000000,0.000000,0.006065,-0.391849
8,2020-12-24,1,0.000000,1.00,MSFT,False,19,0,222.238144,10550600.0,-0.500000,0.000000,-0.005314,-0.533411
13,2021-01-05,2,1.000000,1.00,MSFT,False,25,0,217.399292,23823000.0,1.000000,0.000000,-0.021773,1.257976
14,2021-01-12,1,0.000000,1.00,MSFT,False,24,0,214.436111,23249300.0,-0.500000,0.000000,-0.013630,-0.024082
15,2021-01-14,1,0.000000,1.00,MSFT,False,27,0,212.530518,29480800.0,0.000000,0.000000,-0.008887,0.268030


In [55]:
expected_posts = merged['post_count'].mean()
expected_avg_comments = merged['avg_comments'].mean()
expected_volume_change = merged['Volume'].mean()
expected_300_count = merged['Count_Score_300'].mean()

In [56]:
#Live API call to Reddit
def df_from_response(res):
    df = pd.DataFrame()

    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'num_comments': post['data']['num_comments'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'date': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%d'),
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'kind': post['kind'],
            'id': post['data']['id'],
        }, ignore_index=True)
    return df

In [57]:
auth = requests.auth.HTTPBasicAuth(script, secret)
data = {'grant_type': 'password',
        'username': 'NoShare8264',
        'password': 'NewPass227$'}

headers = {'User-Agent': 'Final_Project/0.0.1'}

request = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
token = f"bearer {request.json()['access_token']}"
headers = {**headers, **{'Authorization': token}}

In [58]:
data = pd.DataFrame()

length = 0
# for i in range(10):
try:
    params = {'limit': 100,
              'q': ticker,
              'restrict_sr': True}
    res = requests.get("https://oauth.reddit.com/r/WallStreetBets/search",
                       headers=headers,
                       params=params)

    new_df = df_from_response(res)
    new_df.sort_values(by=['date'], inplace=True, ascending=False, axis=0)
    row = new_df.iloc[len(new_df)-1]
    fullname = row['kind'] + '_' + row['id']
    params['after'] = fullname
    data = data.append(new_df, ignore_index=True)
except:
    data

In [59]:
data

,date,downs,id,kind,num_comments,score,selftext,subreddit,title,ups,upvote_ratio
0,2021-03-07,0.0,lzuorp,t3,26.0,35.0,60k Microsoft customers were found to be hacke...,wallstreetbets,MSFT hack provides CRWD dip buying opp,35.0,0.79
1,2021-02-16,0.0,ll68pc,t3,3547.0,49403.0,"Hey 'Tards,\n\nThe New Failure to deliver dat...",wallstreetbets,The SEC Just posted the new numbers for Failur...,49403.0,0.91
2,2021-02-10,0.0,lgpwez,t3,1244.0,9442.0,"Listen up you fucks, for most of you this is y...",wallstreetbets,Why this is just a beginning: a guide for The ...,9442.0,0.88
3,2021-02-05,0.0,ld15zm,t3,14.0,7.0,So who do you think has the best chance at get...,wallstreetbets,"AAPL + hyundai, GOOG + F, MSFT + GM.. Autonomo...",7.0,0.73
4,2021-02-04,0.0,ld0aqu,t3,0.0,10.0,,wallstreetbets,Some Loss For MSFT,10.0,0.99
...,...,...,...,...,...,...,...,...,...,...,...
95,2020-02-26,0.0,f9r6l1,t3,371.0,3076.0,Stop selling off MSFT.\n\nCOVID is biological ...,wallstreetbets,MSFT panic sellers,3076.0,0.98
96,2020-02-25,0.0,f9ap2m,t3,277.0,1149.0,Our Billy\n\nWho art in Seattle\n\nHallowed be...,wallstreetbets,A Prayer for the MSFT Autists Today,1149.0,0.96
97,2020-02-13,0.0,f3fzoh,t3,531.0,2478.0,"If you have anything less than a March call, y...",wallstreetbets,Here's the truth about $MSFT,2478.0,0.97
98,2020-02-13,0.0,f3f2sk,t3,892.0,4512.0,,wallstreetbets,BREAKING: Court grants Amazon Web Services req...,4512.0,0.95


In [60]:
data.to_csv("test.csv")

In [61]:
data['date'] = pd.to_datetime(data['date'])

In [62]:
super_posts_live = data.groupby('date')['score'].apply(lambda x: (x >= 300).sum()).reset_index(name='Count_Score_300')
if not super_posts_live.empty:
    super_posts_live = super_posts_live
else:
    super_posts_live['Count_Score_300'] = 0

In [63]:

try:
    data['Ticker'] = ticker
    live_group = data.groupby('date') \
       .agg({'id':'count', 'num_comments':'mean', 'score':'mean'}) \
       .rename(columns={'id':'post_count','num_comments':'avg_comments', 'score':'avg_score'}) \
       .reset_index()
    live_group['Ticker'] = ticker
except:
    places = {'created_utc': 0, 'post_count': 0, 'avg_comments': 0, 'avg_score': 0, 'ticker': ticker}
    live_group = pd.DataFrame.from_dict(places)

In [64]:
live_group = live_group.merge(super_posts_live, left_on=['date'], right_on=['date'], how="left")

In [65]:
live_group

,date,post_count,avg_comments,avg_score,Ticker,Count_Score_300
0,2020-02-11,1,325.0,3271.0,MSFT,1
1,2020-02-13,2,711.5,3495.0,MSFT,2
2,2020-02-25,1,277.0,1149.0,MSFT,1
3,2020-02-26,1,371.0,3076.0,MSFT,1
4,2020-04-29,1,214.0,192.0,MSFT,0
...,...,...,...,...,...,...
73,2021-02-04,1,0.0,10.0,MSFT,0
74,2021-02-05,1,14.0,7.0,MSFT,0
75,2021-02-10,1,1244.0,9442.0,MSFT,1
76,2021-02-16,1,3547.0,49403.0,MSFT,1


In [66]:
live_group = live_group.merge(stockReport, left_on=['date', 'Ticker'], right_on=['Date', 'Ticker'], how='left')
live_group['Adj Close_change'] = live_group['Adj Close'].pct_change()
live_group['Volume_change'] = live_group['Volume'].pct_change()
live_group['post_count_change'] = live_group['post_count'].pct_change()
live_group['avg_score_change'] = live_group['avg_score'].pct_change()

In [67]:
xfits = live_group[live_group.date > datetime.now() - pd.to_timedelta("3day")]
xfits_dates = xfits['date']
xfits = xfits[['post_count_change', 'avg_score_change', 'Volume_change', 'Count_Score_300']]

In [68]:
xfits

,post_count_change,avg_score_change,Volume_change,Count_Score_300
77,0.0,-0.999292,0.0,0


In [69]:
xfits_dates

77   2021-03-07
Name: date, dtype: datetime64[ns]

In [70]:
X_fits_scaled = X_scaler.transform(xfits)

In [71]:
X_fits_scaled

array([[-0.31436072, -0.37907254, -0.18464138, -0.14303404]])

In [72]:
social_predictions = model.predict(X_fits_scaled)

In [73]:
social_predictions = pd.DataFrame(social_predictions.reshape(-1,1))

In [84]:
Xnew, _ = make_regression(n_samples=10, n_features=4, noise=0.01, random_state=1)

In [85]:
ynew = model.predict(Xnew)

In [86]:
future_predict_df = pd.DataFrame(ynew.reshape(-1,1))

In [87]:
live_group.sort_values(by=['date'], inplace=True, ascending=False)

In [88]:
future_predict_df

,0
0,-0.073728
1,0.463298
2,0.583449
3,0.422819
4,-0.190736
5,0.610120
6,0.255038
7,-0.047762
8,0.029407
9,-0.248192


In [101]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [104]:
futureDates = pd.date_range(today, periods = 10, freq=us_bd)
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
futureDates = pd.date_range(today, periods = futureDates, freq=us_bd)

TypeError: periods must be a number, got DatetimeIndex(['2021-03-08 19:20:33.584296', '2021-03-09 19:20:33.584296',
               '2021-03-10 19:20:33.584296', '2021-03-11 19:20:33.584296',
               '2021-03-12 19:20:33.584296', '2021-03-15 19:20:33.584296',
               '2021-03-16 19:20:33.584296', '2021-03-17 19:20:33.584296',
               '2021-03-18 19:20:33.584296', '2021-03-19 19:20:33.584296'],
              dtype='datetime64[ns]', freq='C')